Simple approach to check how it all works.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

In [ ]:
TRAINING_DECKS_PATH = '../trainingDecks.json'
TRAINING_GAMES_PATH = '../training_games.csv'

data = pd.read_json(TRAINING_DECKS_PATH, lines=True)

In [3]:
decks = pd.read_csv("./decks.csv", index_col=0)

In [4]:
dummies_heros = pd.get_dummies(decks['Hero'])

In [5]:
decks = decks.drop(columns=['deckName.1', 'Hero'])
decks = pd.concat([decks, dummies_heros], axis=1)
decks = decks.astype('int64')

In [6]:
print(decks.head())

            Abyssal Enforcer  Acherus Veteran  Acidic Swamp Ooze  \
deckName                                                           
deck252103                 0                0                  0   
deck105300                 0                0                  0   
deck822100                 0                1                  0   
deck829295                 0                0                  0   
deck627915                 0                0                  0   

            Acolyte of Agony  Acolyte of Pain  Al'Akir the Windlord  Alleycat  \
deckName                                                                        
deck252103                 0                2                     0         0   
deck105300                 0                0                     0         0   
deck822100                 0                0                     0         0   
deck829295                 0                2                     0         0   
deck627915                 0         

In [7]:
games = pd.read_csv("./training_games.csv", sep=';', index_col=0)

In [8]:
games.head()

,Gracz1,Talia1,Gracz2,Talia2,Wynik
Id,,,,,
100001.0,A1,deck113225,A1,deck731599,PLAYER_0 WON
100002.0,A1,deck694943,A1,deck929572,PLAYER_1 WON
100003.0,A1,deck182567,A1,deck525929,PLAYER_0 WON
100004.0,A1,deck219364,A1,deck757429,PLAYER_1 WON
100005.0,A1,deck826229,A1,deck337123,PLAYER_1 WON


In [9]:
games.shape

(299680, 5)

In [10]:
games.loc[100001]['Talia1']

'deck113225'

In [11]:
zero_data = np.zeros(shape=(0, 2 * decks.shape[1]))

In [12]:
column_names = 2 * list(decks.columns.values)

In [13]:
training_set = pd.DataFrame(zero_data, columns=column_names)

In [14]:
training_set = pd.concat([decks.loc[games.loc[i]["Talia1"]].append(decks.loc[games.loc[i]["Talia2"]])
                          for i in range(100001, 100001 + games.shape[0])], axis=1, ignore_index=True)

In [15]:
training_set = training_set.transpose()

In [58]:
print(training_set.head())

   Abyssal Enforcer  Acherus Veteran  Acidic Swamp Ooze  Acolyte of Agony  \
0                 0                0                  0                 0   
1                 0                0                  2                 0   
2                 0                0                  0                 0   
3                 0                0                  2                 0   
4                 0                0                  0                 0   

   Acolyte of Pain  Al'Akir the Windlord  Alleycat  Aluneth  \
0                0                     0         0        0   
1                0                     0         0        0   
2                0                     0         0        0   
3                0                     0         0        0   
4                0                     0         0        0   

   Ancestral Healing  Animal Companion   ...     Wrath  Druid  Hunter  Mage  \
0                  0                 0   ...         2      1       0     0   


In [16]:
training_set.shape

(299680, 678)

In [17]:
y = games['Wynik']

In [18]:
y.replace(['PLAYER_0 WON', 'PLAYER_1 WON'], [0, 1])

Id
100001.0    0
100002.0    1
100003.0    0
100004.0    1
100005.0    1
100006.0    1
100007.0    0
100008.0    0
100009.0    0
100010.0    0
100011.0    1
100012.0    0
100013.0    0
100014.0    1
100015.0    0
100016.0    1
100017.0    1
100018.0    0
100019.0    1
100020.0    1
100021.0    1
100022.0    0
100023.0    1
100024.0    0
100025.0    0
100026.0    0
100027.0    1
100028.0    1
100029.0    0
100030.0    0
           ..
399651.0    1
399652.0    0
399653.0    0
399654.0    1
399655.0    0
399656.0    1
399657.0    0
399658.0    1
399659.0    1
399660.0    1
399661.0    0
399662.0    0
399663.0    1
399664.0    1
399665.0    1
399666.0    0
399667.0    0
399668.0    1
399669.0    1
399670.0    0
399671.0    1
399672.0    0
399673.0    1
399674.0    0
399675.0    0
399676.0    0
399677.0    1
399678.0    0
399679.0    1
399680.0    1
Name: Wynik, Length: 299680, dtype: int64

In [19]:
X_train, X_test, y_train, y_test = train_test_split(training_set, y, test_size=0.1)

In [20]:
X_train.shape

(269712, 678)

KNN

In [74]:
parameters = {'n_neighbors': [3, 5, 7]}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)

In [21]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
mnb.score(X_test, y_test)

0.6335758142018153